In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [2]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=False
)

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [4]:
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=50,
    do_sample=True
)

Device set to use cuda


In [5]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

output = generator(prompt)
output

[{'generated_text': " Assure her of safety measures you'll implement. Suggest a date for a garden safety workshop.\n\nDear Sarah,\n\nI am writing to express my sincerest apologies for the unfortunate incident in your garden"}]

In [6]:
output[0]["generated_text"]

" Assure her of safety measures you'll implement. Suggest a date for a garden safety workshop.\n\nDear Sarah,\n\nI am writing to express my sincerest apologies for the unfortunate incident in your garden"

In [7]:
prompt = "The capital city of France is"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")
input_ids

tensor([[ 450, 7483, 4272,  310, 3444,  338]], device='cuda:0')

In [8]:
model_output = model.model(input_ids)
model_output

BaseModelOutputWithPast(last_hidden_state=tensor([[[-0.3047,  1.1953,  0.2988,  ..., -0.3008,  0.6758,  0.1406],
         [-0.1318,  0.3320,  0.3906,  ...,  0.5703, -0.1494, -0.6172],
         [-0.3809,  0.3535,  0.1396,  ..., -0.7422, -0.5742,  0.4082],
         [ 0.2021,  1.3125,  1.8203,  ..., -0.3574,  0.3887,  0.0000],
         [-0.4199,  0.0549,  0.2324,  ..., -0.0481,  0.3535, -0.2100],
         [-0.7852,  0.4570,  0.6094,  ..., -0.1060,  0.0659, -0.0708]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<MulBackward0>), past_key_values=<transformers.cache_utils.DynamicCache object at 0x785ce04c5010>, hidden_states=None, attentions=None)

In [9]:
lm_head_output = model.lm_head(model_output[0])
lm_head_output[:10]

tensor([[[24.7500, 24.8750, 22.7500,  ..., 19.0000, 19.0000, 19.0000],
         [31.0000, 31.5000, 26.0000,  ..., 25.8750, 25.8750, 25.8750],
         [33.2500, 34.0000, 34.5000,  ..., 29.7500, 29.7500, 29.7500],
         [31.0000, 28.5000, 31.8750,  ..., 25.3750, 25.3750, 25.3750],
         [32.7500, 33.0000, 36.7500,  ..., 28.2500, 28.2500, 28.2500],
         [29.0000, 29.8750, 28.1250,  ..., 20.8750, 20.8750, 20.8750]]],
       device='cuda:0', dtype=torch.bfloat16, grad_fn=<SliceBackward0>)

In [10]:
token_id = lm_head_output[0,-1].argmax(-1)
tokenizer.decode(token_id)

'Paris'

## Caching Test

In [11]:
prompt = "Write a very long email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")

In [12]:
%%timeit -n 1

generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=100,
    use_cache=True
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


4.83 s ± 579 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit -n 1

generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=100,
    use_cache=False
)

31.2 s ± 397 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
